In [ ]:
!# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7

Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1197 (delta 2), reused 3 (delta 1), pack-reused 1191
Receiving objects: 100% (1197/1197), 74.23 MiB | 28.33 MiB/s, done.
Resolving deltas: 100% (517/517), done.


In [ ]:
!unzip "/content/1.zip" -d "/content"

Archive:  /content/1.zip
  inflating: /content/1.mp4          


In [ ]:
import torch
path_model = '/content/drive/MyDrive/bestv7.pt'
path_model2 = '/content/drive/MyDrive/exp_Dropletv7/weights/best.pt'
model = torch.hub.load('/content/yolov7', 'custom', path_model ,force_reload=True, source='local',trust_repo=True)
model_Droplet = torch.hub.load('/content/yolov7', 'custom', path_model2,force_reload=True, source='local',trust_repo=True)

requirements: jedi>=0.16 not found and is required by YOLOR, attempting auto-update...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.3 MB/s eta 0:00:00

requirements: 1 package updated per /content/yolov7/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Adding autoShape... 
Adding autoShape... 


In [ ]:
# class 0 is cell, 1 is droplet
import math
import matplotlib.pyplot as plt

def invest_pp(img,para):
  use = img #.astype(np.float32)
  use = cv2.cvtColor(use,cv2.COLOR_RGB2BGR)
  use = cv2.resize(use,para,interpolation=cv2.INTER_CUBIC)
  return use

def predict(img,model,threshold = 0.5,param=(0,0)):
  detections = model(img)
  check_img = img.copy()
  count = 0
  results = detections.pandas().xyxy[0].to_dict(orient="records")
  #filter
  for result in results:
    conf = result['confidence']
    name = result['name']
    class_ = result['class']
    if name =='Plastic microbeads' and conf > 0.5:
      count+=1
  return count
def find_cnts(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(11,11),0)
    canny_cv = cv2.Canny(gray,650,1150,apertureSize=5,L2gradient= True)
    canny_cv = cv2.dilate(canny_cv, None, iterations=4)
    canny_cv = cv2.erode(canny_cv, None, iterations=3)
    cnts, hierarchy = cv2.findContours(canny_cv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    return cnts
    # return gray,canny_cv,cnts
def pp(img,sizee = (640,640)):
  use = img #.astype(np.float32)
  use = cv2.cvtColor(use,cv2.COLOR_BGR2RGB)
  use = cv2.resize(use,sizee,interpolation=cv2.INTER_CUBIC)
  # use = cv2.resize(use,sizee)
  return use
def vote(arr):
  arr = np.array(arr)
  counts = np.bincount(arr)
  # print(counts)
  # print(np.argmax(counts))
  return np.argmax(counts)

def get_info_Droplets(img,threshold = 0.5):
  use_img = pp(img)
  detections = model_Droplet(use_img)
  results = detections.pandas().xyxy[0].to_dict(orient="records")
  Arr_tensor = []
  for result in results:
    conf = result['confidence']
    name = result['name']
    class_ = result['class']
    if name =='ADroplet' and conf > 0.5:
      tensor = (int(result['xmin']),int(result['ymin']),int(result['xmax']),int(result['ymax']))
      cv2.rectangle(use_img,(int(result['xmin']),int(result['ymin'])),(int(result['xmax']),int(result['ymax'])),(12,158,200),2)
      Arr_tensor.append(tensor)
  return Arr_tensor, len(Arr_tensor)

In [ ]:
import cv2
import numpy as np
from google.colab import drive
from sklearn import metrics
import pandas as pd
import os
from google.colab.patches import cv2_imshow

top_left_box = [248,200]#200
bottom_right_box = [385,500]#400

id = 0
video_path = '/content/1.mp4'
cap = cv2.VideoCapture(video_path)

cl = [(0,0,0),(0,255,0),(0,0,255)] #black, green, red
color_ = cl[2]
Arr_count = []
Sign_frameOrigin = True

frame_width = 640
frame_height = 640
frame_rate = 30

count_frame = 0

# Tạo đối tượng VideoWriter để lưu video
video_output_filename = '8_frame.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_output_filename, fourcc, frame_rate, (frame_width, frame_height))


numberCells = None


if (cap.isOpened()== False):
    print("Error opening video file")
while cap.isOpened():
    ret, img = cap.read()
    if ret: # có ảnh là show
      frame_sub = img[250:900,525:1265]
      frame_sub = cv2.resize(frame_sub, (640, 640))
      img_sub = frame_sub.copy()#don't draw in here

      cv2.putText(frame_sub, f'{numberCells}',(300,180),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2, cv2.LINE_AA)

      cv2.rectangle(frame_sub, (top_left_box[0],top_left_box[1]),(bottom_right_box[0],bottom_right_box[1]), color_, 2)
      Arr_in4,len_Arrin4 = get_info_Droplets(img_sub)

      check = 0

      for i in range(0,len_Arrin4):
        x1,y1,x2,y2 = Arr_in4[i][0],Arr_in4[i][1],Arr_in4[i][2],Arr_in4[i][3]
        # print(Arr_in4[i])
        # print("And")
        if (y1 > top_left_box[1] and y2 < bottom_right_box[1]):
          cv2.rectangle(frame_sub, (x1,y1),(x2,y2), cl[1], 2)
          c_x = (x1+x2)/2
          c_y = (y1+y2)/2
          h_ = y2-y1
          x1_new = int(c_x - h_/2)
          x2_new = int(c_x + h_/2)
          y1_new = int(c_y - h_/2)
          y2_new = int(c_y + h_/2)
          use_predImg = img_sub[y1_new:y2_new,x1_new:x2_new]
          used_pred = pp(use_predImg)
          count = predict(used_pred,model,param=(x2_new-x1_new,y2_new-y1_new))
          # print(count)
          Arr_count.append(count)
          # cv2_imshow(use_predImg)
          check = 1

      # cv2_imshow(img)
      # print(len_Arrin4)

      if(check == 0 and Arr_count != []): # trong 1 frame khong co giot nao trong vung lam viec
        Arr_count_7 = Arr_count[0:8]
        numberCells = vote(Arr_count_7)
        if len(Arr_count_7)>=8:
          print(Arr_count_7)
          Arr_count = []
          id = id + 1
          print(id)

      out.write(frame_sub)
      if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    else:
        break
cap.release()
out.release()

[7, 6, 5, 5, 5, 5, 5, 5]
1
[5, 5, 2, 2, 2, 2, 2, 2]
2
[8, 8, 9, 9, 9, 9, 8, 8]
3
[2, 0, 0, 2, 0, 0, 0, 0]
4
[3, 3, 3, 4, 4, 3, 3, 3]
5
[2, 1, 1, 1, 1, 1, 1, 1]
6
[3, 3, 3, 2, 2, 2, 3, 3]
7
[0, 0, 0, 0, 0, 0, 0, 0]
8
[4, 4, 4, 2, 2, 2, 2, 2]
9
[2, 2, 1, 1, 1, 2, 1, 1]
10
[2, 1, 1, 1, 1, 1, 1, 1]
11
[3, 2, 2, 2, 2, 3, 2, 2]
12
[2, 2, 2, 2, 2, 2, 2, 2]
13
[2, 1, 1, 1, 1, 1, 1, 1]
14
[1, 1, 1, 0, 0, 1, 0, 0]
15
[0, 0, 0, 0, 0, 1, 0, 0]
16
[2, 1, 1, 1, 2, 2, 1, 1]
17
[1, 1, 0, 0, 0, 0, 0, 1]
18
[1, 1, 1, 1, 2, 2, 1, 1]
19
[2, 2, 0, 1, 1, 1, 1, 1]
20
[0, 0, 1, 1, 1, 1, 1, 1]
21
[1, 1, 0, 1, 1, 2, 1, 1]
22
[1, 1, 0, 0, 0, 0, 0, 0]
23
[1, 1, 1, 0, 0, 0, 0, 0]
24
[2, 2, 2, 2, 2, 2, 2, 2]
25
[1, 1, 1, 1, 1, 1, 1, 1]
26
[1, 1, 1, 1, 1, 1, 1, 1]
27
[0, 0, 0, 0, 0, 0, 0, 0]
28
[1, 1, 1, 1, 1, 2, 2, 1]
29
[0, 0, 0, 0, 0, 0, 0, 0]
30
[1, 0, 0, 0, 0, 0, 0, 0]
31
[1, 1, 1, 0, 0, 0, 0, 0]
32
[1, 1, 1, 1, 1, 1, 1, 1]
33
[1, 1, 0, 1, 1, 0, 0, 1]
34
[0, 0, 0, 0, 0, 0, 0, 0]
35
[0, 0, 0, 1, 0, 0, 0, 0]
36
[